<a href="https://colab.research.google.com/github/nehahimane/COMP_6721_Project_GroupS/blob/AI_Hardik/model_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/AI_Datasets"

In [ ]:
%cd /content/drive/MyDrive/AI_Datasets

/content/drive/MyDrive/AI_Datasets


In [ ]:
!kaggle datasets download -d nightfury007/fercustomdataset-3classes

100% 83.0M/83.1M [00:00<00:00, 159MB/s]
100% 83.1M/83.1M [00:00<00:00, 142MB/s]


In [ ]:
import zipfile
for file in os.listdir():
    if file.endswith(".zip"):
        with zipfile.ZipFile(file, "r") as zip_file:
            zip_file.extractall()
        os.remove(file)

In [ ]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
import glob
import torch.optim as optim
import time
import cv2
import torchvision.datasets as datasets
%matplotlib inline

In [ ]:
data_dir = '/content/drive/MyDrive/AI_Datasets/FER_Custom_Dataset'
IMAGE_DIM=224

print(os.listdir(data_dir))

['Disappointed', 'interested', 'neutral']


In [ ]:
def load_data(path, test_split, batch_size, input_size, shuffle_dataset=True,random_seed=42):
    
    transform_dict ={'src': transforms.Compose([transforms.Resize(IMAGE_DIM),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                     std=[0.229, 0.224, 0.225]
                                                                    )
                                               ]
                                              )
                    }
    
    dataset = datasets.ImageFolder(data_dir, transform=transform_dict.get('src'))
    dataset_size = len(dataset)
    train_test_indices = list(range(dataset_size))
    test_split = int(np.floor(test_split * dataset_size))
    
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(train_test_indices)
    train_indices, test_indices = train_test_indices[test_split:], train_test_indices[:test_split]
    

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    data_loader_train = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                              sampler=train_sampler)
    data_loader_test = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    sampler=test_sampler)
    
    return data_loader_train, data_loader_test

In [ ]:
train_loader, test_loader=load_data(data_dir,0.2,32,(64,64))

print(len(train_loader), len(test_loader))

1096 274


In [ ]:
######## Write your code here ##############
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=False)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

num_epochs = 1
total_steps = len(train_loader)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        images, labels = data[0].to(device), data[1].to(device)
        print(i)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cpu
0
1
2
3
4
5
6
7
8
9
Epoch [1/1], Step [10/1096], Loss: 5.7729, Accuracy: 18.75%
10
11
12
13
14
15
16
17
18
19
Epoch [1/1], Step [20/1096], Loss: 2.0750, Accuracy: 21.88%
20
21
22
23
24
25
26
27
28
29
Epoch [1/1], Step [30/1096], Loss: 1.3226, Accuracy: 28.12%
30
31
32
33
34
35
36
37
38
39
Epoch [1/1], Step [40/1096], Loss: 1.0701, Accuracy: 46.88%
40
41
42
43
44
45
46
47
48
49
Epoch [1/1], Step [50/1096], Loss: 1.1453, Accuracy: 31.25%
50
51
52
53
54
55
56
57
58
59
Epoch [1/1], Step [60/1096], Loss: 1.1285, Accuracy: 34.38%
60
61
62
63
64
65
66
67
68
69
Epoch [1/1], Step [70/1096], Loss: 1.1802, Accuracy: 25.00%
70
71
72
73
74
75
76
77
78
79
Epoch [1/1], Step [80/1096], Loss: 1.1267, Accuracy: 34.38%
80
81
82
83
84
85
86
87
88
89
Epoch [1/1], Step [90/1096], Loss: 1.1361, Accuracy: 25.00%
90
91
92
93
94
95
96
97
98
99
Epoch [1/1], Step [100/1096], Loss: 1.0870, Accuracy: 43.75%
100
101
102
103
104
105
106
107
108
109
Epoch [1/1], Step [110/1096], Loss: 1.1118, Accuracy: 34.